In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Earlier we made recommendation engine using content of movies(content based filtering) but we found out that it was not able 
to capture the taste(user likings and tastes)so we want some system(algorithm)that will capture the user taste and will recommend the similar
movies to the user as per their taste(collaborative filtering approach)

One of the major drawback which we kind of feel from content based filtering in case of movies recommendation that it is not person wise centered means anyone finding similar movies to some movie will get the same results.

It is basically of two types user:

User based filtering:This system recommend products to the users which similar users have liked(user similarity concept-can use user information for grouping categories).Finding out the correlation(Using pearson correlation or say the cosine similarity).Although computing user-based CF is very simple, it suffers from several problems. One main issue is that users preference changes over time. It indicates that precomputing the matrix based on their neighboring users may lead to bad performance.To get over this problem, we can apply item-based CF.

Item based filtering:Instead of measuring the similarity between users,the item-based CF recommends items based on their similarity with the items that the target user has rated.We will check the similarity within our items.

In [2]:
#Reading the dataset
df = pd.read_csv("ratings_small.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
#Getting the info about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


Link-https://surprise.readthedocs.io/en/stable/getting_started.html
INfo about using surprise library

In [4]:
#Surprise has a set of built-in algorithms and datasets to play with.In its simplest form,it only takes a few lines of code to run a cross-validation procedure
from surprise import Reader,Dataset,SVD,accuracy
reader = Reader()

In [5]:
#Here rating are on a scale of 5
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [6]:
#We will use the SVD algorithm.
algo = SVD()
#Run 5-fold cross-validation and print results
from surprise.model_selection import cross_validate
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8971  0.8861  0.9005  0.9037  0.8963  0.8967  0.0060  
MAE (testset)     0.6885  0.6809  0.6931  0.6983  0.6904  0.6902  0.0057  
Fit time          4.28    4.33    4.24    4.27    4.48    4.32    0.08    
Test time         0.21    0.14    0.14    0.21    0.19    0.18    0.03    


{'test_rmse': array([0.8971492 , 0.88605709, 0.90045071, 0.90373381, 0.89626222]),
 'test_mae': array([0.68847263, 0.68093558, 0.69307463, 0.69829029, 0.69043283]),
 'fit_time': (4.277029037475586,
  4.331018447875977,
  4.241023540496826,
  4.272989273071289,
  4.4794275760650635),
 'test_time': (0.20974969863891602,
  0.1448359489440918,
  0.13531017303466797,
  0.2108309268951416,
  0.1865241527557373)}

In [7]:
#Splitting the data
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25)
#Training the algorithm on the trainset,and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
#Then computing RMSE
accuracy.rmse(predictions)

RMSE: 0.9050


0.9049832731484546

In [8]:
#Now just picking a user id and checking what ratings should have been been given by user(or has given)
df[df["userId"]==1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [9]:
algo.predict(1,2105,4)
#For the movie id we get an estimated prediction of 3.08..
#It tries to predict ratings based on how the other users have predicted the movie

Prediction(uid=1, iid=2105, r_ui=4, est=2.883193139678455, details={'was_impossible': False})